In [22]:
import os
from Bio import AlignIO
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from Bio.Align import MultipleSeqAlignment, SeqRecord

import seaborn as sns

from collections import defaultdict

In [23]:
NUMBER = 1
FILE = f'data/seed_{NUMBER}/align_seed_{NUMBER}.fasta'

In [24]:
align = AlignIO.read(FILE, format='fasta')
all_records = align._records
align

<<class 'Bio.Align.MultipleSeqAlignment'> instance (187 records of length 437) at 12d180c48b0>

In [25]:
def filter(record, *names): # нет названий (names) в таксономии
    for name in names:
        if name in record.description:
            return False
    return True

def get_level(record, num):
    return record.description.split("/")[num]

`get_level(record, -3)` - описание последовательности выглядит так (посмотрите в vscode сам файлик .fasta, можно будет на структуру посмотреть файлика): `<taxonomy>/<id>`

`<id>` - последние два значения через слеши, их убрать надо и взять только название вида:

<strike>cellular organisms/Bacteria/FCB group/Bacteroidota/Chlorobiota group/Bacteroidota/Bacteroidia/Marinilabiliales/Prolixibacteraceae/</strike>Draconibacterium<strike>/X5DE21_9BACT/199-424</strike>


In [26]:
records = [SeqRecord(record.seq, 
                     f'{get_level(record, -3)}/{get_level(record, -2)}/{get_level(record, -1)}', 
                     name='', description='') for record in align._records if filter(record, 'Fungi', 'Bacteria')]
len(records)

61

In [27]:
records_only_fungi = [SeqRecord(record.seq, 
                                f'{get_level(record, -3)}/{get_level(record, -2)}/{get_level(record, -1)}', 
                                name='', description='') for record in align._records if not filter(record, 'Fungi')]
len(records_only_fungi)

118

In [28]:
records_only_bacteria = [SeqRecord(record.seq, 
                                   f'{get_level(record, -3)}/{get_level(record, -2)}/{get_level(record, -1)}', 
                                   name='', description='') 
                                   for record in align._records if not filter(record, 'Bacteria')]
len(records_only_bacteria)


8

In [29]:
all_records = records + records_only_fungi + records_only_bacteria
all_records = MultipleSeqAlignment(all_records)
all_records.sort()

all_records

<<class 'Bio.Align.MultipleSeqAlignment'> instance (187 records of length 437) at 12d18171660>

In [30]:
AlignIO.write(all_records, open(f'data/seed_{NUMBER}/align_seed_names_{NUMBER}.fasta', "w"), "fasta")

1